### Calculate the metrics

1. F1
2. Mean IoU
3. Recall
4. Precision
5. Accuracy

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.metrics import f1_score, jaccard_score, recall_score, precision_score, accuracy_score

### Load predict mask and ground truth mask

In [4]:
# Paths to predicted masks and ground truth masks
pred_mask_dir = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/predicted_masks/aug"
true_mask_dir = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/processed_dataset/aug/test/masks"

In [5]:
# Load mask file paths
pred_mask_paths = sorted(glob(os.path.join(pred_mask_dir, "*.png")))
true_mask_paths = sorted(glob(os.path.join(true_mask_dir, "*.png")))

# Ensure equal number of files
assert len(pred_mask_paths) == len(true_mask_paths), "Mismatch in predicted and ground truth mask counts."

### Calculate the metrics

In [6]:
# Updated class mappings for grayscale values
CLASS_MAPPING = {0: 0, 128: 1, 255: 2}
VALID_VALUES = np.array([0, 128, 255])  # Expected grayscale values

# Function to convert grayscale values to class indices
def convert_mask(mask):
    mask_converted = np.zeros_like(mask, dtype=np.uint8)
    for gray_value, class_id in CLASS_MAPPING.items():
        mask_converted[mask == gray_value] = class_id
    return mask_converted

# Initialize metrics
scores = []

# Metrics Calculation
for pred_path, true_path in tqdm(zip(pred_mask_paths, true_mask_paths), total=len(pred_mask_paths), desc="Evaluating"):
    # Load predicted and true masks
    pred_mask = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)
    true_mask = cv2.imread(true_path, cv2.IMREAD_GRAYSCALE)

    if pred_mask is None or true_mask is None:
        print(f"Skipping {pred_path} or {true_path} due to loading error.")
        continue

    # Validate mask values
    if not np.all(np.isin(np.unique(pred_mask), VALID_VALUES)) or not np.all(np.isin(np.unique(true_mask), VALID_VALUES)):
        print(f"Unexpected values in {pred_path} or {true_path}. Skipping...")
        continue

    # Convert grayscale mask values to class indices (0, 1, 2)
    pred_mask = convert_mask(pred_mask).flatten().astype(np.int32)
    true_mask = convert_mask(true_mask).flatten().astype(np.int32)

    # Calculate metrics
    acc = accuracy_score(true_mask, pred_mask)
    f1 = f1_score(true_mask, pred_mask, average="macro", labels=[0, 1, 2])
    iou = jaccard_score(true_mask, pred_mask, average="macro", labels=[0, 1, 2])
    recall = recall_score(true_mask, pred_mask, average="macro", labels=[0, 1, 2])
    precision = precision_score(true_mask, pred_mask, average="macro", labels=[0, 1, 2])

    # Append metrics
    scores.append([os.path.basename(pred_path), acc, f1, iou, recall, precision])

# Check if any valid scores were collected
if len(scores) == 0:
    print("No valid masks evaluated.")
    exit()


Evaluating:  68%|██████▊   | 13/19 [00:20<00:09,  1.59s/it]/home/ahsan/PortableApps/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Evaluating:  74%|███████▎  | 14/19 [00:22<00:07,  1.59s/it]/home/ahsan/PortableApps/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Evaluating: 100%|██████████| 19/19 [00:30<00:00,  1.60s/it]


In [7]:
# Convert to numpy array for mean calculation
scores_np = np.array([s[1:] for s in scores])

In [9]:
mean_scores = np.mean(scores_np, axis=0)


In [10]:
# Print Metrics
print(f"Mean Accuracy: {mean_scores[0]:0.5f}")
print(f"Mean F1 Score: {mean_scores[1]:0.5f}")
print(f"Mean IoU: {mean_scores[2]:0.5f}")
print(f"Mean Recall: {mean_scores[3]:0.5f}")
print(f"Mean Precision: {mean_scores[4]:0.5f}")

Mean Accuracy: 0.94043
Mean F1 Score: 0.85517
Mean IoU: 0.80693
Mean Recall: 0.85777
Mean Precision: 0.87467


In [11]:
# Optional: Save detailed metrics to a file
output_file = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/aug/evaluation_results.csv"
with open(output_file, "w") as f:
    f.write("Image,Accuracy,F1 Score,IoU,Recall,Precision\n")
    for score in scores:
        f.write(",".join(map(str, score)) + "\n")
    f.write(f"\nMean Metrics,,{mean_scores[0]:0.5f},{mean_scores[1]:0.5f},{mean_scores[2]:0.5f},{mean_scores[3]:0.5f},{mean_scores[4]:0.5f}\n")
    
print(f"Metrics saved to {output_file}")

Metrics saved to /home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/aug/evaluation_results.csv


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load evaluation results from CSV
csv_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/aug/evaluation_results.csv"
df = pd.read_csv(csv_path)

# Compute mean scores for visualization
mean_scores = df.iloc[-1, 2:].values.astype(float)

metrics = ["Accuracy", "F1 Score", "IoU", "Recall", "Precision"]
values = list(mean_scores)

# Bar chart for metric comparison
plt.figure(figsize=(10, 6))
plt.bar(metrics, values, color=['blue', 'green', 'orange', 'red', 'purple'])
plt.ylim(0, 1)
plt.xlabel("Metrics")
plt.ylabel("Score")
plt.title("Evaluation Metrics")
plt.grid(axis="y", linestyle="--", alpha=0.7)
# plt.xticks(rotation=45)
# Display the chart
plt.show()
